# Install Necessary Libraries for the Project

In [7]:
# !conda config --append channels pytorch
# !conda install -y datasets accelerate trl rapidfuzz
# !conda install -y conda-forge::bert_score
!conda install -y conda-forge::evaluate

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /users/0/brogn002/miniconda3/envs/main

  added / updated specs:
    - conda-forge::evaluate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiofiles-22.1.0            |  py312h06a4308_0          26 KB
    altair-5.5.0               |  py312h06a4308_0         798 KB
    annotated-types-0.6.0      |  py312h06a4308_0          25 KB
    binaryornot-0.4.4          |     pyhd3eb1b0_1         351 KB
    chardet-4.0.0              |py312h06a4308_1003         217 KB
    click-8.1.7                |  py312h06a4308_0         203 KB
    cookiecutter-2.6.0         |  py312h06a4308_0         139 KB
    evaluate-0.4.3             |     pyh29332c

# Download the dataset from Kaggle and Preprocess it

In [1]:
import os
import kagglehub
import pandas as pd

# Download the dataset from Kaggle
path = kagglehub.dataset_download("shanegerami/ai-vs-human-text")
print("Path to dataset files:", path)

# Convert the .csv dataset file into a Pandas dataframe
df = pd.read_csv(os.path.join(path, "AI_Human.csv"))

# Remove the 'generated' column to save space
df = df.drop(columns=["generated"])

# Display the first few rows of the dataframe
print("Number of rows in the dataframe:", len(df))
df.head()

100%|████████████████████████████████████████| 350M/350M [00:05<00:00, 73.0MB/s]

Extracting model files...


Path to dataset files: /users/0/brogn002/.cache/kagglehub/datasets/shanegerami/ai-vs-human-text/versions/1
Number of rows in the dataframe: 487235


,text
0,Cars. Cars have been around since they became ...
1,Transportation is a large necessity in most co...
2,"""America's love affair with it's vehicles seem..."
3,How often do you ride in a car? Do you drive a...
4,Cars are a wonderful thing. They are perhaps o...


In [3]:
import random

# Make all of the text lowercase and strip leading and trailing spaces
df["text"] = df["text"].str.lower().str.strip()

# Filter out examples containing non-ASCII characters
def is_valid_example(text):
    if '\\' in r"%r" % text:
        return False
    if '#' in text:
        return False
    try:
        text.encode('ascii')
        return True
    except UnicodeEncodeError:
        return False
df_sample = df[df["text"].apply(is_valid_example)]
print(f"Number of valid ascii examples: {len(df_sample)}")

# Only use 5000 examples from the dataset
df_sample = df_sample.sample(n=5000)
print("Number of rows in the sampled dataframe:", len(df_sample))

# Take a random number of sentences from each sample
for i in range(len(df_sample)):
    df_sample["text"].iloc[i] = (".".join(df_sample["text"].iloc[i].split(".")[:random.randint(2, 6)])) + "."

original_alphabet = 'abcdefghijklmnopqrstuvwxyz'
# Function to generate a random mono-alphabetic substitution cipher
def generate_cipher():
    shuffled = random.sample(list(original_alphabet), len(original_alphabet))
    return ''.join(shuffled)
# Function to encrypt text using a generated cipher
def encrypt_text(text):
    cipher = generate_cipher()
    translation_table = str.maketrans(original_alphabet, cipher)
    encrypted = text.translate(translation_table)
    return encrypted, cipher

# Apply encryption and add new columns
df_sample[['encrypted_text', 'cipher']] = df_sample['text'].apply(encrypt_text).apply(pd.Series)

# Save the processed dataframe to disk
processed_data_path = 'processed_dataset.csv'
df_sample.to_csv(processed_data_path, index=False)
print(f"Processed dataset saved to {processed_data_path}")

# Display the first few rows of the dataframe
df_sample.head()

Number of valid ascii examples: 11801
Number of rows in the sampled dataframe: 5000


/tmp/ipykernel_1283178/2588153823.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_sample["text"].iloc[i] = (".".join(df_sample["text"].iloc[i].split(".")[:random.randint(2, 6)])) + "."


Processed dataset saved to processed_dataset.csv


,text,encrypted_text,cipher
151828,there are pros and cons to learning to compete...,cousu gsu jsmr gbi kmbr cm eugsbabw cm kmtjucu...,gvkiuqwoafnetbmjzsrcdxpyhl
122945,the picture of faces on mars are actually natu...,and pbvahwd xo oqvdu xk fqwu qwd qvahqeej kqah...,qcvsdoznbmyefkxprwuahigtjl
368391,i think the facial action coding system is val...,g cugqe cus mlfgld lfcgvq fvagqz nincsw gn jld...,lbfasmzugxedwqvyhoncrjkpit
453015,i agree that it's difficult to change impressi...,y zgrkk wnzw yw'a eyffybxuw wd bnzvgk yjorkaay...,zlbekfgnyicujvdoprawxtqmhs
262365,driverless cars may be modern and use half of ...,hlboylcyuu xilu fip wy fkhylr irh nuy dicv kv ...,iwxhyvedbtjcfrkzglumnoaspq


# Train a model on the Dataset object

In [6]:
%%writefile train_grpo.py

import os
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import AutoModelForCausalLM
from datasets import Dataset

# Load the preprocessed dataset
processed_data_path = 'processed_dataset.csv'
df = pd.read_csv(processed_data_path)

# Turn the Pandas df into a Dataset object
# Leave 100 examples out of the training set for testing
train_dataset = Dataset.from_pandas(df[100:])

# Create prompts for GRPO Training
def create_prompt(example):
    example["prompt"] = f"""You are an expert cryptanalyst specialized in solving monoalphabetic substitution ciphers. Your task is to decode encrypted messages and provide the plaintext solution. When given an encrypted message, follow these steps:
          1. Analyze letter frequencies in the ciphertext
          2. Find one letter words (Such as 'i' or 'a')
          3. Identify common patterns and letter combinations
          4. Map cipher letters to plaintext letters
          5. Refine your mapping until the message is coherent
          6. Return only the decrypted text with no additional commentary
          Remember that in a monoalphabetic cipher, each letter in the original text is replaced by a different letter consistently throughout the message. For example, if 'a' is encrypted as 'x', then every 'a' in the plaintext will appear as 'x' in the ciphertext. Provide only the decrypted plaintext as your response without any explanation or additional text. Here is a message encrypted using a monoalphabetic substitution cipher. Every english character is mapped to a different english character, but all punctuation remains the same: {example['encrypted_text']}
          What is the unencrypted English plaintext:"""
    return example
train_dataset = train_dataset.map(create_prompt)
print(train_dataset)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training will run on: {device}")

# Create a unique checkpoint directory for each run using a timestamp
run = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
checkpoint_dir = f'/users/0/brogn002/{run}'
os.makedirs(checkpoint_dir, exist_ok=True)

from trl import GRPOConfig, GRPOTrainer
from rapidfuzz import fuzz

# Define a reward function to use
class DynamicWeightReward:
    def __init__(self, dataset_size, batch_size, epochs):
        """Initialize the reward tracker with training parameters"""
        self.__name__ = "dynamic_weight_reward"
        self.step = 0
        self.max_steps = (dataset_size // batch_size) * epochs
        print(f"DynamicWeightReward initialized with {self.max_steps} estimated steps")
    
    def __call__(self, completions, **kwargs):
        """
        Callable reward function that computes similarity scores with 
        weights that shift from length focus to accuracy focus over time.
        """
        original_texts = kwargs["text"]
        
        # Increment step counter
        self.step += 1
        
        # Calculate progress as a value between 0 and 1
        progress = min(self.step / max(self.max_steps, 1), 1.0)
        
        # Calculate dynamic weights based on training progress
        # Start with 90% length, 10% accuracy
        # End with 10% length, 90% accuracy
        length_weight = 0.9 - (0.8 * progress)
        accuracy_weight = 1.0 - length_weight
        
        # Log the current weights periodically
        if self.step % 100 == 0:
            print(f"Step {self.step}/{self.max_steps}: Length weight = {length_weight:.2f}, Accuracy weight = {accuracy_weight:.2f}")
        
        rewards = []
        for completion, text in zip(completions, original_texts):
            # Clean up completion to remove any potential formatting
            completion = completion.strip().lower()
            text = text.strip().lower()
            
            # Do not reward empty strings
            if len(completion) == 0:
                rewards.append(0.0)
                continue
                
            # Do not reward answers with non-ascii characters
            try:
                completion.encode('ascii')
            except UnicodeEncodeError:
                rewards.append(0.0)
                continue
                
            # Perfect match gets a full reward
            if completion == text:
                rewards.append(1.0)
                continue
                
            # Apply RapidFuzz ratio for accuracy component
            similarity = fuzz.ratio(completion, text) / 100.0
            
            # Calculate length component
            length_penalty = max(0, 1 - (abs(len(completion) - len(text)) / max(len(text), 1)))
            
            # Combine with dynamic weights
            final_score = (similarity * accuracy_weight) + (length_penalty * length_weight)
            
            rewards.append(final_score)
            
        return rewards

# # Path to the pretrained model to start from
# model_dir = '/users/0/brogn002/PHI_LETTER_SWAP_2025-04-06_23-14-08/final_model'
# print(os.listdir(model_dir)) # Verify that the folder exists

num_epochs = 3
training_args = GRPOConfig(
    output_dir=checkpoint_dir,
    logging_steps=50,
    per_device_train_batch_size=4,  # Decrease this to lower vram usage
    num_generations=4,  # Decrease this to lower vram usage
    save_strategy="no",  # Do not save checkpoints (saves storage space)
    bf16=True,  # Enable bf16 mixed precision on A100 GPUs
    num_train_epochs=num_epochs,
)

reward_func = DynamicWeightReward(
    dataset_size=len(train_dataset),
    batch_size=training_args.per_device_train_batch_size,
    epochs=num_epochs
)

trainer = GRPOTrainer(
    model="microsoft/Phi-4-mini-instruct",
    reward_funcs=reward_func,  # Pass the callable reward class instance
    args=training_args,
    train_dataset=train_dataset,
)

# Train and save the final model
trainer.train()
trainer.save_model(os.path.join(checkpoint_dir, "final_model"))

Overwriting train_grpo.py


In [14]:
!accelerate launch --config_file deepspeed_zero3.yaml train_grpo.py

[2025-04-06 11:28:55,130] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
df: /users/0/brogn002/.triton/autotune: No such file or directory
Traceback (most recent call last):
  File "/users/0/brogn002/miniconda3/envs/main/bin/accelerate", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/launch.py", line 1177, in launch_command
    deepspeed_launcher(args)
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/launch.py", line 826, in deepspeed_launcher
    from deepspeed.launcher.runner import DEEPSPEED_ENVIRONMENT_NAME
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/deepspeed/__init__.py", line 25, in <module>
    

# Load the Trained Model and Test it Against the Baseline

In [2]:
import os
import evaluate
import bert_score
import statistics
import torch
from tqdm import tqdm
import numpy as np
from rapidfuzz import fuzz
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the preprocessed dataset
processed_data_path = 'processed_dataset.csv'
df = pd.read_csv(processed_data_path)

# Turn the Pandas df into a Dataset object
# 100 examples were left out of the training set for testing
test_dataset = Dataset.from_pandas(df[:100])

# Create prompts for GRPO Training
def create_prompt(example):
    example["prompt"] = f"""Two of the letters in the following message have been swapped. Please determine which two letters were swapped and give me the original message. Do not output any explanation or additional text beyond the original message. Here is the message to solve with the two swapped letters: {example["encrypted_text"]}"""
    return example
test_dataset = test_dataset.map(create_prompt)
print(test_dataset)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Evaluation running on: {device}")

# Path to the checkpoints directory
checkpoint_dir = '/users/0/brogn002/2025-04-11_19-13-56/final_model'
print(os.listdir(checkpoint_dir)) # Verify that the folder exists
# Load fine-tuned model using local_files_only and from_pretrained
finetuned_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=checkpoint_dir,
    local_files_only=True,
    device_map="auto"  # Support distributed training
)
finetuned_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=checkpoint_dir,
    local_files_only=True
)

# Load baseline model
baseline_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    device_map="auto"  # Support distributed training
)
baseline_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")

finetuned_predictions = []
base_predictions = []
references = []

# Set models to evaluation mode
finetuned_model.eval()
baseline_model.eval()

# Disable gradient computation for inference
with torch.no_grad():
    for example in tqdm(test_dataset, desc="Evaluating Models"):
        encrypted_text = example['encrypted_text']
        original_text = example['text']
        prompt = example['prompt']

        # Prepare inputs for fine-tuned model
        ft_inputs = finetuned_tokenizer(prompt, return_tensors="pt")
        ft_inputs = {k: v.to(device) for k, v in ft_inputs.items()}

        # Prepare inputs for baseline model
        baseline_inputs = baseline_tokenizer(prompt, return_tensors="pt")
        baseline_inputs = {k: v.to(device) for k, v in baseline_inputs.items()}

        # Generate decryption for fine-tuned model
        ft_outputs = finetuned_model.generate(
            **ft_inputs,
            max_new_tokens=len(encrypted_text),
            num_return_sequences=1
        )

        # Generate decryption for baseline model
        baseline_outputs = baseline_model.generate(
            **baseline_inputs,
            max_new_tokens=len(encrypted_text),
            num_return_sequences=1
        )

        # Decode outputs
        ft_decoded = finetuned_tokenizer.decode(ft_outputs[0], skip_special_tokens=True)
        baseline_decoded = baseline_tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)

        # Clean up outputs (remove prompt)
        ft_decoded = ft_decoded.replace(prompt, '').strip()
        baseline_decoded = baseline_decoded.replace(prompt, '').strip()

        finetuned_predictions.append(ft_decoded)
        base_predictions.append(baseline_decoded)
        references.append(original_text)

        print(f"\nOriginal Text: {original_text}")
        print(f"\nCipher Text: {encrypted_text}")
        print(f"\nFine-Tuned Model Output: {ft_decoded}")
        print(f"\nBaseline Model Output: {baseline_decoded}")

# Compute overall metrics
bleu = evaluate.load("bleu")
bleu_baseline = bleu.compute(predictions=base_predictions, references=references)
bleu_finetuned = bleu.compute(predictions=finetuned_predictions, references=references)

bertscore = evaluate.load("bertscore")
bert_baseline = bertscore.compute(predictions=base_predictions, references=references, lang="en")
bert_finetuned = bertscore.compute(predictions=finetuned_predictions, references=references, lang="en")

# Print detailed results
print("\nModel Evaluation Results:")
print(f"Fine-tuned Model - Mean Bleu Score: {statistics.mean(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Mean Bleu Score: {statistics.mean(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Median Bleu Score: {statistics.median(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Median Bleu Score: {statistics.mean(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Bleu Score Std Dev: {statistics.stdev(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Bleu Score Std Dev: {statistics.stdev(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Mean Bert Score: {statistics.mean(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Mean Bert Score: {statistics.mean(bert_baseline['precision']):.4f}")
print(f"Fine-tuned Model - Median Bert Score: {statistics.median(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Median Bert Score: {statistics.mean(bert_baseline['precision']):.4f}")
print(f"Fine-tuned Model - Bert Score Std Dev: {statistics.stdev(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Bert Score Std Dev: {statistics.stdev(bert_baseline['precision']):.4f}")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'encrypted_text', 'cipher', 'prompt'],
    num_rows: 100
})
Evaluation running on: cuda
['added_tokens.json', 'config.json', 'generation_config.json', 'merges.txt', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'model.safetensors.index.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.json']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating Models:   1%|▏                       | 1/100 [00:17<28:40, 17.38s/it]

...


Original Text: excerpt: having a positive attitude ij essential to leading a successful and happy life. it ij important to continually cultivate a positive outlook in all aspects of life: professional, personal, and family. it can help up achieve our goal and provide up with the confidence and optimism necessary to reach our full potential. positive attitude can have a profound impact on our performance in the workplace and on our relationship with other.

Cipher Text: iboidsw: tfphlc f symhwhpi fwwhwkji hn immilwhfg wy gifjhlc f mkooimmekg flj tfssu ghei. hw hn hqsydwflw wy oylwhlkfggu okgwhpfwi f symhwhpi ykwgyya hl fgg fmsiowm ye ghei: sdyeimmhylfg, sidmylfg, flj efqhgu. hw ofl tigs ks fothipi ykd cyfg flj sdyphji ks rhwt wti oylehjiloi flj yswhqhmq lioimmfdu wy difot ykd ekgg sywilwhfg. symhwhpi fwwhwkji ofl tfpi f sdyeyklj hqsfow yl ykd sideydqfloi hl wti rydasgfoi flj yl ykd digfwhylmths rhwt ywtid.

Fine-Tuned Model Output: 

Baseline Model Output: fwwhwkji ofl tfpi f sdyeyklj 

Evaluating Models:  13%|██▉                    | 13/100 [06:19<51:28, 35.50s/it]


Original Text: i think huge experienced wonderful and amazing things he also may have seen disturbing things like seeing the acropolis in greece that was probably amazing but seeing war results was probably very disturbing. i also think traveling months at a time would make me really anxious and board until the games came in then that would have made a little less bearing, but i still would have been a little anxious to get to where i needed to go. and i couldn't imagine going across the atlantic ocean 16 times and the pacific ocean twice that is just absolutely crazy i really don't know how he huge merger and his friend don last. i did not know who these men were or what this story was about, but after i read this story i now highly respect huge merger and his friend don last for what they did and how many times they risked their lives to save other people that were in trouble..

Cipher Text: z mazul acvo owgoszouyoh fkuhosice ruh rnrjzuv mazuvd ao redk nrx arto doou hzdmcsbzuv mazuv

Evaluating Models:  13%|██▉                    | 13/100 [06:34<44:00, 30.35s/it]


KeyboardInterrupt: 